In [4]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from transformer.column_transformer import ColumnTransformer
from transformer.date_transformer import DateTransformer
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math

In [5]:
## Loading Constants
start = "2008-01-01"
end = datetime.now().strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("unity")
market = Market()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
reload = True
model_range = range(len(tickers))
datasets = [
                "pdr"
            ,"tiingo"
            ,"finnhub"
           ]

In [6]:
refined_regression = []
refined_daily_classification = []
refined_quarterly_classification = []
refined_weekly_classification = []
market.connect()
if reload:
    for dataset in datasets:
        for i in tqdm(model_range):
            try:
                ticker = tickers["Symbol"][i]
                if "." in ticker:
                    ticker = ticker.replace(".","-")
                prices = market.retrieve_price_data("{}_prices".format(dataset),ticker)
                if dataset == "pdr":
                    prices = ColumnTransformer.rename_columns(prices, " ")
                else:
                    prices = ColumnTransformer.rename_columns(prices, "_")
                prices = DateTransformer.convert_to_date(dataset,prices,"date")
                prices.reset_index(inplace=True)
                relev = prices[["date","adjclose"]]
    #             relev["date"] = [datetime.strptime(x.split("T")[0],"%Y-%m-%d") for x in relev["date"]]
                relev["ticker"] = ticker
                relev["week"] = [x.week for x in relev["date"]]
                relev["quarter"] = [x.quarter for x in relev["date"]]
                relev["year"] = [x.year for x in relev["date"]]
                relev.sort_values("date",inplace=True)
                ## daily transformations
                refined_regression.append(relev)
                relev_classification = relev.copy()
                relev_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_classification["adjclose"].diff()]
                refined_daily_classification.append(relev_classification)
                ## weekly transformations
                relev["week"] = [x.week for x in relev["date"]]
                relev["quarter"] = [x.quarter for x in relev["date"]]
                relev["year"] = [x.year for x in relev["date"]]
                relev_weekly_classification = relev.groupby(["year","quarter","week"]).mean().reset_index()
                relev_weekly_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_weekly_classification["adjclose"].diff()]
                relev_weekly_classification["ticker"] = ticker
                refined_weekly_classification.append(relev_weekly_classification)
                ## quarterly transformations
                relev_quarterly_classification = relev.groupby(["year","quarter"]).mean().reset_index().drop("week",axis=1)
                relev_quarterly_classification["adjclose"] = [1 if x > 0 else 0 for x in relev_quarterly_classification["adjclose"].diff()]
                relev_quarterly_classification["ticker"] = ticker
                refined_quarterly_classification.append(relev_quarterly_classification)
            except Exception as e:
                print(str(e),ticker)
        classification_sets = {"date":refined_daily_classification,
                               "quarter":refined_quarterly_classification,
                              "week":refined_weekly_classification}
        for ds in classification_sets:
            base = pd.concat(classification_sets[ds])
            if ds == "date":
                base["year"] = [x.year for x in base["date"]]
                base["quarter"] = [x.quarter for x in base["date"]]
                base["week"] = [x.week for x in base["date"]]
                final = base.pivot_table(index=ds,values="adjclose",columns="ticker").reset_index()
            else:
                if ds == "week":
                    final = base.pivot_table(index=["year","quarter","week"],values="adjclose",columns="ticker").reset_index()
                else:
                    final = base.pivot_table(index=["year",ds],values="adjclose",columns="ticker").reset_index()
            name = "dataset_{}_{}_classification".format(dataset,ds)
            final.fillna(-99999,inplace=True)
            market.drop_table(name)
            market.store_data(name,final)
        base = pd.concat(refined_regression)
        final = base.pivot_table(index=["year","quarter","week"],values="adjclose",columns="ticker").reset_index()
        final.fillna(-99999,inplace=True)
        for timeframe in ["week","quarter"]:
            if timeframe == "week":
                relev = final.groupby(["year","quarter","week"]).mean().reset_index()
            else:
                relev = final.groupby(["year","quarter"]).mean().reset_index()
            relev.reset_index(drop=True,inplace=True)
            name = "dataset_{}_{}_regression".format(dataset,timeframe)
            market.drop_table(name)
            market.store_data(name,relev)

  0%|                                                                                                                                              | 0/506 [00:00<?, ?it/s]<ipython-input-6-35e10f3e184b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relev["ticker"] = ticker
<ipython-input-6-35e10f3e184b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relev["week"] = [x.week for x in relev["date"]]
<ipython-input-6-35e10f3e184b>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

't' CXO


 88%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                | 443/506 [05:05<00:06,  9.91it/s]

't' TIF


 24%|███████████████████████████████▊                                                                                                    | 122/506 [00:15<00:42,  8.94it/s]

...

't' MET
't' MGM


 88%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                | 443/506 [00:49<00:05, 10.54it/s]

't' TIF


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 506/506 [00:55<00:00,  9.04it/s]


't' ZBRA
't' ZBH
't' ZION
't' ZTS
